In [1]:
import numpy as np
dataset_filename = 'affinity_dataset.txt'
table = np.loadtxt(dataset_filename)

In [4]:
# 面包、牛奶、奶酪、苹果、香蕉
print(table[: 5])

[[ 0.  1.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  1.]
 [ 1.  1.  0.  0.  0.]
 [ 0.  0.  1.  1.  1.]]


In [ ]:
# 计算规则“如果顾客购买了苹果，他们也会购买香蕉”的支持度（support）和置信度（confidence）
# 支持度：指数据集中规则应验的次数
# 置信度：符合给定条件（即规则的“如果”语句所表示的前提条件）的所有规则里，跟当前规则结论一致的比例
# 规则：一条规则由前提条件和结论两部分组成。规则的优劣有多种衡量方法，常见的有支持度和置信度